# Лабораторная работа No 5. DES.
    выполнил Сергей Харитонов ПВ-32 (Вариант 14)

# Цель работы
    освоить алгоритм шифра DES

In [1]:
# Начальная перестановка
P_start = [
    58, 50, 42, 34, 26, 18, 10, 2,
    60, 52, 44, 36, 28, 20, 12, 4,
    62, 54, 46, 38, 30, 22, 14, 6,
    64, 56, 48, 40, 32, 24, 16, 8,
    57, 49, 41, 33, 25, 17,  9, 1,
    59, 51, 43, 35, 27, 19, 11, 3, 
    61, 53, 45, 37, 29, 21, 13, 5,
    63, 55, 47, 39, 31, 23, 15, 7
]

# Конечная перестановка
P_finish = [
    40,  8, 48, 16, 56, 24, 64, 32,
    39,  7, 47, 15, 55, 23, 63, 31,
    38,  6, 46, 14, 54, 22, 62, 30,
    37,  5, 45, 13, 53, 21, 61, 29,
    36,  4, 44, 12, 52, 20, 60, 28,
    35,  3, 43, 11, 51, 19, 59, 27,
    34,  2, 42, 10, 50, 18, 58, 26,
    33,  1, 41,  9, 49, 17, 57, 25,
]

# Блок расширения
P_extern = [
    32,  1,  2,  3,  4,  5,
     4,  5,  6,  7,  8,  9,
     8,  9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32,  1
]

# Прямой блок
P_smooth = [
    16,  7, 20, 21, 29, 12, 28, 17,
     1, 15, 23, 26,  5, 18, 31, 10,
     2,  8, 24, 14, 32, 27,  3,  9,
    19, 13, 30,  6, 22, 11,  4, 25
]

# Массив S блоков
S_arr = [
    [
        [14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
        [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
        [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
        [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]
    ],
    [
        [15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10],
        [ 3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5],
        [ 0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15],
        [13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9]
    ],
    [
        [10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8],
        [13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1],
        [13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7],
        [ 1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12]
    ],
    [
        [ 7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
        [13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
        [10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
        [ 3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14]
    ],
    [
        [ 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
        [14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6],
        [ 4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
        [11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]
    ],
    [
        [12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
        [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
        [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
        [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]
    ],
    [
        [ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
        [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
        [ 1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2],
        [ 6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12]
    ],
    [
        [13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
        [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
        [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
        [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]
    ],
]

# массив для получения 56 битного ключа из 64
P_get_key = [
    57, 49, 41, 33, 25, 17,  9,  1, 58, 50, 42, 34, 26, 18,
    10,  2, 59, 51, 43, 35, 27, 19, 11,  3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,  7, 62, 54, 46, 38, 30, 22,
    14,  6, 61, 53, 45, 37, 29, 21, 13,  5, 28, 20, 12,  4
]

# массив говорящий на сколько нужно сдвигать CD
circle_cd_arr = [
    1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1
]

# блок для получения ключа из CD
P_get_key_from_cd = [
    14, 17, 11, 24,  1,  5,  3, 28, 15,  6, 21, 10, 23, 19, 12,  4,
    26,  8, 16,  7, 27, 20, 13,  2, 41, 52, 31, 37, 47, 55, 30, 40,
    51, 45, 33, 48, 44, 49, 39, 56, 34, 53, 46, 42, 50, 36, 29, 32

]

        Создать подпрограмму, которая выполняет функцию P-блока, прямого и расширения, используя таблицы перестановок. Подпрограмма принимает блок данных и соответствующую таблицу (начальной перестановки, конечной перестановки или P-блока расширения), возвращает после операции блок данных.

In [2]:
def P(data_block: list, P_table: list) -> list:
    return [data_block[index-1] for index in P_table]

        Создать подпрограмму - отбеливатель, которая выполняет функцию отбеливания двух равных блоков данных (XOR). Подпрограмма принимает два блока данных, и возвращает блок данных, который является результатом проведения операции XOR (исключающее или) между двумя принятыми блоками.

In [3]:
def whitener(data_block_f: list, data_block_s: list) -> list:
    return [data_block_f[index] ^ data_block_s[index] for index in range(len(data_block_f))]

        Создать подпрограмму, которая выполняет функцию S-блока, используя таблицы смешивания. Подпрограмма принимает блок данных, массив таблиц (правила для S-блоков), после операции смешивания возвращает блок данных.

In [4]:
# напишем пару вспомогательных функций
def get_s_cord(data_block: list) -> (int, int):
    # Возвращает координаты числа в S таблице по блоку данных из 6ти битов
    return (data_block[0]*2 + data_block[-1],
            data_block[1]*8 + data_block[2]*4 + data_block[3]*2 + data_block[4])

def get_bit_list(number: int, extern_to:int = 4) -> list:
    # Возвращает массив битов числа, дополняет нулями до extern_to битов
    result = []
    while number:
        result = [number & 1] + result
        number >>= 1
        
    if len(result) < extern_to:
        result = [0]*(extern_to-len(result)) + result
        
    return result

def bits_to_hex(bits: list) -> str:
    result = 0
    for bit in bits:
        result |= bit
        result <<= 1
    result >>= 1
    return hex(result)[2:]

def S(data_block: list, S_arr:list) -> list:
    #Выполняет S блок
    result = []
    for n in range(8):
        start_pos = n*6
        piece_block = data_block[start_pos: start_pos+6]
        
        s_cord = get_s_cord(piece_block)
        number = S_arr[n][s_cord[0]][s_cord[1]]
        
        result += get_bit_list(number)
    return result
        

        Создать подпрограмму, которая реализует основную функцию DES, с помощью подпрограмм, описанных в предыдущих пунктах. Подпрограмма принимает блок данных длиной 32 бита и 48-битный ключ. Функция DES содержит P-блок расширения, отбеливатель, группу S-блоков и прямой P-блок. Подпрограмма возвращает 32-битовое слово.

In [5]:
def F(data_block: list, key: list, show_process: bool = False):
    extern = P(data_block, P_extern)
    xored  = whitener(extern, key)
    sed    = S(xored, S_arr)
    result = P(sed, P_smooth)
    
    if show_process:
        print(f"Ключ:              {bits_to_hex(key)}")
        print(f"На входе:          {bits_to_hex(data_block)}")
        print(f"После расширения:  {bits_to_hex(extern)}")
        print(f"После отбеливания: {bits_to_hex(xored)}")
        print(f"После смешивания:  {bits_to_hex(sed)}")
        print(f"Результат:         {bits_to_hex(result)}")
    
    return result

        С помощью подпрограммы, реализующей основную функцию DES, зашифровать блок данных соответствующий варианту c помощью ключа. Показать, как изменялся блок данных на каждом из этапов. Блок данных представляет собой четыре байта в шестнадцатеричном виде, ключ шесть байтов.

In [6]:
data_block = get_bit_list(int("C0F537D1", 16), 32)
key        = get_bit_list(int("B2EBA2363BED", 16), 48)

f_result   = F(data_block, key, True)

Ключ:              b2eba2363bed
На входе:          c0f537d1
После расширения:  e017aa9afea3
После отбеливания: 52fc08acc54e
После смешивания:  62b0e651
Результат:         47348b8c


        Создать подпрограмму, которая принимает блок данных и разбивает его на два равных.

In [7]:
def split_data_block(data_block: list) -> (list, list):
    center = len(data_block)//2
    return (data_block[:center], data_block[center:])

        Создать подпрограмму, которая принимает блок данных и число, циклически сдвигает содержимое влево на принятое число.

In [8]:
def l_circle(data_block: list, count: int) -> list:
    new_data_block = data_block.copy()
    for _ in range(count):
        first = new_data_block.pop(0)
        new_data_block.append(first)
    return new_data_block

        Создать подпрограмму которая реализует алгоритм генерации ключей раунда, используя подпрограмму P-блока из предыдущей лабораторной работы, а также подпрограммы, реализованные в предыдущих пунктах. Таблица удаления проверочных битов, таблица, которая содержит число сдвигаемых битов в раунде и таблица сжатия ключа находится в приложении.

In [9]:
def gen_keys(key_for_key: list) -> list:
    key56 = P(key_for_key, P_get_key)
    cd_arr = [key56]
    for shift in circle_cd_arr:
        left, right = split_data_block(cd_arr[-1])
        left  = l_circle(left, shift)
        right = l_circle(right, shift)
        cd_arr.append(left+right)
    cd_arr = cd_arr[1:]
    return [P(cd, P_get_key_from_cd) for cd in cd_arr]

        Выполнить задание своего варианта. С помощью созданной подпрограммы генерации ключей и данного в варианте ключа сгенерировать ключи раунда.

In [10]:
key_for_key = get_bit_list(int("85BD31AC83EC69EC", 16), 64)
keys = gen_keys(key_for_key)
for i in range(16):
    print(f"Ключ номер {i+1}: {bits_to_hex(keys[i])}")

Ключ номер 1: 43717521587c
Ключ номер 2: 4ee47dda4e50
Ключ номер 3: cfc51a99e318
Ключ номер 4: 6e8babb17600
Ключ номер 5: bbb80bf82226
Ключ номер 6: 292edab46a8e
Ключ номер 7: 747c9c3432d3
Ключ номер 8: 56ad50b7a063
Ключ номер 9: d2697563438f
Ключ номер 10: 8de72556118b
Ключ номер 11: c317afc6116d
Ключ номер 12: f992e142bbe8
Ключ номер 13: 91daea709d39
Ключ номер 14: b073d64b1c3a
Ключ номер 15: 345f454d7938
Ключ номер 16: 6e3d509cce21


        На основе предыдущих лабораторных работ реализовать алгоритм шифрования DES.

In [11]:
def DES(data_block: list, key_for_keys: list):
    keys = gen_keys(key_for_keys)
    start_shape = P(data_block, P_start)
    tmp_data_block = start_shape
    for key in keys:
        left, rigth = split_data_block(tmp_data_block)
        f_value = F(rigth, key)
        
        new_rigth = whitener(left, f_value)
        new_left  = rigth
#         print(f"Левая часть: {bits_to_hex(new_left)}, Правая часть: {bits_to_hex(new_rigth)}, Ключ: {bits_to_hex(key)}")
        tmp_data_block = new_left + new_rigth
    rigth, left = split_data_block(tmp_data_block)
    finish_shape = P(left+rigth, P_finish)
    return finish_shape

        Программно реализовать режим шифрования ECB.

In [12]:
def ECB(hex_data: str, key_for_keys: list):
    result_data = ""
    while(hex_data):
        cur_data_block = get_bit_list(int(hex_data[:16], 16), 64)
        hex_data = hex_data[16:]
        
        coded_data_block = DES(cur_data_block, key_for_keys)
        
        result_data += bits_to_hex(coded_data_block)
    return result_data

    Программно реализовать режим шифрования CBC.

In [13]:
def CBC(hex_data: str, key_for_keys: list, hex_c0: str):
    c = get_bit_list(int(hex_c0, 16), 64)
    result_data = ""
    while(hex_data):
        cur_data_block = get_bit_list(int(hex_data[:16], 16), 64)
        hex_data = hex_data[16:]
        
        cur_data_block = whitener(cur_data_block, c)
        coded_data_block = DES(cur_data_block, key_for_keys)
        c = coded_data_block
        
        result_data += bits_to_hex(coded_data_block)
    return result_data

        Программно реализовать режим шифрования CFB.

In [14]:
def CFB(hex_data: str, key_for_keys: list, hex_c0: str, j: int):
    c = get_bit_list(int(hex_c0, 16), 64)
    bit_data = get_bit_list(int(hex_data, 16))
    result_data = []
    
    while(bit_data):
        c   = DES(c, key_for_keys)
        c_j = c[:j]
        
        bit_data_j = bit_data[:j]
        bit_data   = bit_data[j:]
#         расширяем до размера c если надо
        if (len(c_j) > len(bit_data_j)):
            bit_data_j = [0]*(len(c_j) - len(bit_data_j)) + bit_data_j
        
        code_bit_data_j = whitener(c_j, bit_data_j)
        result_data += code_bit_data_j
        
        c = [0]*(64 - len(code_bit_data_j)) + code_bit_data_j
        
    return bits_to_hex(result_data)

        Зашифровать текст своего варианта каждым из режимов шифрования.

In [15]:
import binascii
  
def str2hex(s):
    return binascii.hexlify(bytes(str.encode(s))).decode('utf-8')
 
data_str = "Дайтечеловекувсё,чегоонжелает,ивтужеминутуонпочувствует,чтоэтовсёнеестьвсё."
key_for_key_hex = "835F06B30AC1031D"
c0_hex = "3837363534333231"
j = 64

data_hex = str2hex(data_str)
key_for_key = get_bit_list(int(key_for_key_hex, 16), 64)

ECB_hex = ECB(data_hex, key_for_key)
CBC_hex = CBC(data_hex, key_for_key, c0_hex)
CFB_hex = CFB(data_hex, key_for_key, c0_hex, j)

print(f"Кодируемая строка: {data_str}")
print(f"Кодируемая строка в HEX: {data_hex}")
print(f"Вектор начальной инициализации в HEX: {c0_hex}")
print()
print(f"Ключ в HEX: {key_for_key_hex}")
print(f"Ключ в битах: {key_for_key}")
print()
print(f"ECB в HEX: {ECB_hex}")
print()
print(f"CBC в HEX: {CBC_hex}")
print()
print(f"CFB в HEX: {CFB_hex}")

Кодируемая строка: Дайтечеловекувсё,чегоонжелает,ивтужеминутуонпочувствует,чтоэтовсёнеестьвсё.
Кодируемая строка в HEX: d094d0b0d0b9d182d0b5d187d0b5d0bbd0bed0b2d0b5d0bad183d0b2d181d1912cd187d0b5d0b3d0bed0bed0bdd0b6d0b5d0bbd0b0d0b5d1822cd0b8d0b2d182d183d0b6d0b5d0bcd0b8d0bdd183d182d183d0bed0bdd0bfd0bed187d183d0b2d181d182d0b2d183d0b5d1822cd187d182d0bed18dd182d0bed0b2d181d191d0bdd0b5d0b5d181d182d18cd0b2d181d1912e
Вектор начальной инициализации в HEX: 3837363534333231

Ключ в HEX: 835F06B30AC1031D
Ключ в битах: [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]

ECB в HEX: b70b5ad57d07e6c5aefc4e5b1a238204ec49af8e33eb08adb886e58b743d31e9a44bf5ab1708895680b2645b858d69925709b229e07b56926c4aa069526ce4d8699a7c58d02bbda8944457e2d37ffe2732629cadef1c96564c5d4fe802e65e3972fb3e3cc78cb6d0ad98bc8534f9c06c5c12f4a740faab418a152a1b145dbc1917b54109f517834b1c33a230f3e